In [1]:
import os
from util import data
from pdb import set_trace
from multiprocessing import pool

%load_ext autoreload
%autoreload 2

default_pool_size = max(1, os.cpu_count()-2)

music21: Certain music21 functions might need these optional packages: matplotlib, scipy;
                   if you run into errors, install them by following the instructions at
                   http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
scores = data.load_data()

Serialized scores found, loading...
Scores loaded in 26.69 seconds.


In [3]:
dataset = data.HaydnDataset(data=scores)

Building dataset...
Finished building dataset in 7.59 seconds.


In [4]:
states = list(filter(lambda ds: ds.shape[1] != 0, dataset))
total_ticks = sum(map(lambda state: state.shape[1], states))
print("There are {} final corpi with a total of {} ticks.".format(len(states), total_ticks))

There are 81 final corpi with a total of 144076 ticks.


In [310]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # useful stateless functions
import numpy as np
from torch.utils import data

In [311]:
class Music_Gen_Model(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size , center_input_size, center_output_size, output_size):
        super(Music_Gen_Model, self).__init__()
        
        #TODO: Could make lastm as stacked models with layers=2
        self.left_lstm = nn.LSTM(lstm_input_size, lstm_hidden_size)
        self.right_lstm = nn.LSTM(lstm_input_size, lstm_hidden_size)
        self.center_affine = nn.Linear(center_input_size, center_output_size)
        self.pred_affine = nn.Linear(lstm_hidden_size * 2 + center_output_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, left_seq, center_input, right_seq):
        
        T, N, D = left_seq.size()
        
        '''
        input_size = 66*3
        LSTM input shape: [sqd_len, batch=1, input_size]
        1 = num_layers * num_directions
        LSTM hn output shape: [1, batch, hidden_size]
        
        center_input_size = 66*3
        centerAffine input shape: [batch, center_input_size]
        centerAffine output shape: [batch, center_output_shape]
        
        '''        
        left_out, (left_hn, left_cn) = self.left_lstm(left_seq)
        right_out, (right_hn, right_cn) = self.right_lstm(right_seq)
        
        center_out = self.center_affine(center_input)
        
        left_hn = left_hn.view(N, -1)
        right_hn = right_hn.view(N, -1)
        
        '''
        left_hn shape: [batch, hidden_size]
        right_hn shape: [batch, hidden_size]
        center_out shape: [batch, center_out_size]
        '''
        merge_input = torch.cat((left_hn, center_out, right_hn), 1)
        pred_output = self.pred_affine(merge_input)
        output = self.relu(pred_output)
        output = self.softmax(output)
        
        return output
    
#     def initHidden(self):
#         return torch.zeros(1, self.hidden_size, dtype=torch.long)
        

In [312]:
def train(model, left_tensor, right_tensor, center_tensor, target_tensor, optimizer):
        
    output = model(left_tensor, center_tensor, right_tensor)

    target = target_tensor[:, 0:65]
    target_index_tensor = (target.flatten()==1).nonzero()[0]
#     print(target_index_tensor)
    loss = F.nll_loss(output, target_index_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    return output, loss.item()

In [313]:
def generateMiniBatchFromOneMusicForPart(music_data, seq_length, part):
    '''    
    input: music_data. A numpy array with shape [4, tick, 66]
    return: a list contains miniBatch data for one part. Each item in the miniBatch list contains:
            - left_tensor: shape [seq_length, 1, D]
            - right_tensor: shape [seq_length, 1, D]
            - center_tensor: shape [1, D]
            - target_tensor: shape [1, 66]
        D = size of array contains the data from three other parts
        
    TODO: (1) tempo is currently not added
        
    '''
    miniBatches = []
    
    music_length = music_data.shape[1]
    i = seq_length # TODO: maybe add functinality to pad the beginning and end.
    
    while (i + seq_length) < music_length:
        left_range_start = i-seq_length
        left_range_end = i
        center_index = i
        right_range_start = i + 1
        right_range_end = right_range_start + seq_length
        
        target_result = music_data[part, i+1, :]
        target_tensor = torch.from_numpy(target_result.reshape(1, -1)).float()
        
        left_range = music_data[:, left_range_start:left_range_end, :]
        left_result = np.delete(left_range, part, axis=0)
        left_result = np.swapaxes(left_result, 0, 1)
        seq_l, part_l, notes_l = left_result.shape
        left_tensor = torch.from_numpy(left_result.reshape(seq_l, 1, part_l*notes_l)).float()
        
        right_range = music_data[:, right_range_start:right_range_end, :]
        right_result = np.delete(right_range, part, axis=0)     
        right_result = np.swapaxes(right_result, 0, 1)
        seq_r, part_r, notes_r = right_result.shape    
        right_tensor = torch.from_numpy(right_result.reshape(seq_r, 1, part_r*notes_r)).float()
        
        center_result = music_data[:, i, :]
        center_result = np.delete(center_result, part, axis=0)
        part_c, notes_c = center_result.shape
        center_tensor = torch.from_numpy(center_result.reshape(1, part_c * notes_c)).float()
        
        miniBatches.append((left_tensor, right_tensor, center_tensor, target_tensor))
        
        i += 1
        
    return miniBatches
    

In [314]:
'''
mini generateMiniBatchFromOneMusicForPart test
'''
# fake_data = np.random.rand(4, 7, 3)
# r = generateMiniBatchFromOneMusicForPart(fake_data, 3, 0)
# left_tensor, right_tensor, center_tensor, target_tensor = r[0]
# print(left_tensor.shape)

# print('fake_data')
# print(fake_data)

'\nmini generateMiniBatchFromOneMusicForPart test\n'

In [315]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [316]:
lstm_input_size = 66 * 3
lstm_hidden_size = 200
center_input_size = 66 * 3
center_output_size = 100
output_size = 66

n_iters = 2000
print_every = 100
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters
learning_rate = 0.005


model_Pt1 = Music_Gen_Model(lstm_input_size, lstm_hidden_size, center_input_size, center_output_size, output_size)
model_Pt2 = Music_Gen_Model(lstm_input_size, lstm_hidden_size, center_input_size, center_output_size, output_size)
model_Pt3 = Music_Gen_Model(lstm_input_size, lstm_hidden_size, center_input_size, center_output_size, output_size)
model_Pt4 = Music_Gen_Model(lstm_input_size, lstm_hidden_size, center_input_size, center_output_size, output_size)


optimizer_pt1 = optim.SGD(model_Pt1.parameters(), lr=learning_rate)
optimizer_pt2 = optim.SGD(model_Pt2.parameters(), lr=learning_rate)
optimizer_pt3 = optim.SGD(model_Pt3.parameters(), lr=learning_rate)
optimizer_pt4 = optim.SGD(model_Pt4.parameters(), lr=learning_rate)


seq_length = 8
demo_piece = dataset.__getitem__(3) #numpy array
print(demo_piece.shape)
demo_piece_minibatch_pt1 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 0)
demo_piece_minibatch_pt2 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 1)
demo_piece_minibatch_pt3 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 2)
demo_piece_minibatch_pt4 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 3)
miniBatch_length = len(demo_piece_minibatch_pt1)
print("minibatch list length: {x}".format(x=miniBatch_length))

#Training for Part 1
start = time.time()
ite = 0
'''
IMPORTANT: Model, Optimizer, and demo_miniBatch must match!!!
'''

'''
NOTE: This is only trainning for part 1, with data from one music piece.

TODO:
(1) generate minibatch from other pieces
(2) train for other parts
(3) write sampling function

'''
for n in range(n_iters):
    if ite/miniBatch_length >= 1:
        ite = 0
            
    left_tensor, right_tensor, center_tensor, target_tensor = demo_piece_minibatch_pt1[ite]

#     print("train left_tensor size: {x}".format(x=left_tensor.size()))
    output, loss = train(model_Pt1, left_tensor, right_tensor, center_tensor, target_tensor, optimizer_pt1)
    total_loss += loss
    
    if (n+1) % print_every == 0:
            print('%s (%d %d%%) loss: %.4f' % (timeSince(start), n, n / n_iters * 100, loss))
            print("output: {x}".format(x=output))
            
    if (n+1) % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0
    
    ite += 1
    n += 1


(4, 1541, 66)
minibatch list length: 1525


/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0m 0s (99 4%) loss: -0.0148
output: tensor([[0.0163, 0.0151, 0.0152, 0.0160, 0.0148, 0.0148, 0.0153, 0.0151, 0.0154,
         0.0155, 0.0148, 0.0148, 0.0148, 0.0148, 0.0152, 0.0148, 0.0148, 0.0161,
         0.0148, 0.0148, 0.0148, 0.0161, 0.0148, 0.0148, 0.0156, 0.0148, 0.0148,
         0.0154, 0.0154, 0.0158, 0.0148, 0.0149, 0.0149, 0.0162, 0.0151, 0.0161,
         0.0151, 0.0159, 0.0148, 0.0150, 0.0148, 0.0155, 0.0157, 0.0153, 0.0148,
         0.0153, 0.0151, 0.0148, 0.0148, 0.0148, 0.0156, 0.0148, 0.0150, 0.0148,
         0.0148, 0.0148, 0.0148, 0.0148, 0.0149, 0.0154, 0.0148, 0.0157, 0.0148,
         0.0156, 0.0148, 0.0148]], grad_fn=<SoftmaxBackward>)
0m 1s (199 9%) loss: -0.0149
output: tensor([[0.0166, 0.0158, 0.0149, 0.0153, 0.0149, 0.0149, 0.0149, 0.0149, 0.0149,
         0.0155, 0.0149, 0.0149, 0.0149, 0.0149, 0.0149, 0.0153, 0.0156, 0.0158,
         0.0149, 0.0158, 0.0149, 0.0153, 0.0149, 0.0149, 0.0149, 0.0149, 0.0149,
         0.0154, 0.0159, 0.0154, 0.0149, 0.0149, 0.0160

KeyboardInterrupt: 